In [1]:
%pip install cryptography
%pip install pyopenssl


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

def encrypt_with_public_key(public_key_path, plaintext):
    # Load public key from file
    with open("public_key.pem", "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )

    # Encrypt plaintext with the public key
    ciphertext = public_key.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return ciphertext

def decrypt_with_private_key(private_key_path, ciphertext):
    # Load private key from file
    with open("private_key.pem", "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )

    # Decrypt ciphertext with the private key
    decrypted_text = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return decrypted_text

# Test the functions
if __name__ == "__main__":
    plaintext = b"This is a message to be encrypted."
    
    encrypted_text = encrypt_with_public_key("path_to_public_key.pem", plaintext)
    print("Encrypted:", encrypted_text)

    decrypted_text = decrypt_with_private_key("path_to_private_key.pem", encrypted_text)
    print("Decrypted:", decrypted_text.decode())


Encrypted: b'UHW\xee\x0f\xce\xf8\xcd\x0c\xcf\x9f\xbcrz\xea\x1f\xe1\x9bH\x05\xdb\xfd\x90Z\x1af\xbd\xc8\x83\x9d\xc8\xb0\x9c3}j[J\xfc\xe3\xa7\x9f)\xe1^H4\xc5X\xf5\x9c&\x92\xb0{\xd3\xa4\xa52\xeey\xa9q\xa8\xf6\xa5\x8dJ\xab\xf2fe\xf2\xe5\x0b\x05\x89\xc4\x9f\x8f\xe8V\xff\x99\xf6\xaf\x16+\x1e\x98\x07\xa0=\xec\x15\xfe=\x12\xff\x88\xed\x93\x88\x86L\x8aT\xb3\x94\xa5\x05\xe5\x19,m\x84\xf4\x8fE\xee\xbc\xb0cN\r\x0bpP\x9f\xf8\xa6\xe6s\xfd\xab\x03\xd2\x99mxw\xb8\xa9}o\x81z\x0b\xc8\xa4&\x04\x9f\xab[\xd8\x8bs\xfdt\x10m\xfc\xf3\xbd\xa6\xf0j^\x01\x89[\xccA\x01\xb5\xb6\xddn\xafT\x0f7\xee\xe7\x95-*\x15\xfc\xfben\xe8D\x83\xa0\xa4XB.\xab-\xc8\xe3B\x93\x8ai,\x97s\x85+\x94\x0c\x84U\xe5\xe5\x01\xf2\xadY\x9f\xadV\x07c\x84\x85\xb5EJ-\xd8\x88\x86b\x8c\xac~\xfbp\x9eL\xec`"\xdc=<p\xf9\xde\xb9'
Decrypted: This is a message to be encrypted.


In [1]:
%pip install cryptography psycopg2


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import psycopg2
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

# 1. 生成公钥和私钥
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# 2. 序列化公钥和私钥，以便存储或传输
pem_private = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)

pem_public = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# 3. 使用公钥加密数据
plaintext = "Hello, World!"
ciphertext = public_key.encrypt(
    plaintext.encode(),
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 4. 将加密的数据存储到 PostgreSQL 数据库
conn = psycopg2.connect(
        host="db_container",  # 这是Docker容器的名字
        dbname="postgres",
        user="postgres",
        password="mysecretpassword"  # 这应该与您启动数据库容器时设置的密码匹配
    )
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS encrypted_data (id serial PRIMARY KEY, data bytea);")
cur.execute("INSERT INTO encrypted_data (data) VALUES (%s) RETURNING id;", (ciphertext,))
row_id = cur.fetchone()[0]
conn.commit()

# 5. 从数据库读取加密的数据
cur.execute("SELECT data FROM encrypted_data WHERE id = %s;", (row_id,))
encrypted_data = cur.fetchone()[0]

# 6. 使用私钥解密数据
decrypted_data = private_key.decrypt(
    encrypted_data,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
).decode()

print(decrypted_data)  # 应该输出 "Hello, World!"

cur.close()
conn.close()


OperationalError: could not translate host name "db_container" to address: Unknown host


In [11]:
from cryptography import x509

In [20]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
import base64

# 生成三个公私钥对
key_pairs = [rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend()) for _ in range(3)]

# 保存公钥和私钥到文件
for i, key_pair in enumerate(key_pairs, 1):
    # 保存公钥
    public_key = key_pair.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open(f"public_key_{i}.pem", "wb") as pub_file:
        pub_file.write(public_key)

    # 保存私钥
    private_key = key_pair.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open(f"private_key_{i}.pem", "wb") as priv_file:
        priv_file.write(private_key)

# 使用私钥生成三个签名，这里我们以公钥作为要签名的数据，并保存到文件
for i, key_pair in enumerate(key_pairs, 1):
    public_key = key_pair.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    signature = key_pair.sign(
        public_key,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    encoded_signature = base64.b64encode(signature)
    
    with open(f"signature_{i}.txt", "w") as sig_file:
        sig_file.write(encoded_signature.decode())


In [2]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import datetime

# Generate a 2048 bit RSA private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

public_key = private_key.public_key()

# Various details about who we are. For a self-signed certificate the
# subject and issuer are always the same.
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"CA"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"mysite.com"),
])
cert = x509.CertificateBuilder().subject_name(
    subject
).issuer_name(
    issuer
).public_key(
    public_key
).serial_number(
    x509.random_serial_number()
).not_valid_before(
    datetime.datetime.utcnow()
).not_valid_after(
    # Our certificate will be valid for 10 days
    datetime.datetime.utcnow() + datetime.timedelta(days=10)
).sign(private_key, hashes.SHA256(), default_backend())

# Write our certificate out to disk.
with open("D:\second_party_certificate.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

# Write our private key out to disk.
with open("D:\second_party_private_key.pem", "wb") as f:
    f.write(private_key.private_bytes(
        serialization.Encoding.PEM,
        serialization.PrivateFormat.TraditionalOpenSSL,
        serialization.NoEncryption()))


In [4]:
%pip install hashlib


Could not fetch URL https://pypi.org/simple/hashlib/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/hashlib/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1122)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib
You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [15]:
import hashlib

# Collect personal information
name = input("Please enter your name: ")
address = input("Please enter your address: ")
phone_number = input("Please enter your phone number: ")

# Combine the personal information into one string
personal_info = name + address + phone_number

# Create a SHA-256 hash of the personal information
hash_object = hashlib.sha256(personal_info.encode())
hex_dig = hash_object.hexdigest()

print("The SHA-256 hash of your personal information is: ", hex_dig)


The SHA-256 hash of your personal information is:  5be8db6710c0d553823fd70398ff78cb0d0934af5a9d39bfb943fad6338c5573


In [5]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Load the private key
with open("D:\second_party_private_key.pem", "rb") as key_file:  # replace with your actual private key file path
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # replace with your password if the private key is encrypted
        backend=default_backend()
    )

# The corresponding public key can be obtained from the private key
public_key = private_key.public_key()

# Serialize the public key in PEM format
public_key_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

print(public_key_pem.decode())


-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAu/II+Mkik7dLKZ2DjAOm
HfBILLvocD8K6xTK8XxbTnuurggcNsdFH5Lt53BrHKAXQJXXgamyFMPnRULnzjAw
aS/MHYF1ui/Q2jJPX7fB4pt97OZWrkxWu7UtyeBgkGNsid9YZN1wK17xibFG87ih
ifrrFS3wudtghDxm1ye/Vcb8S20hu0eP5436t107L1metVpRt7SB6jVvRD1/Jkpt
ouPuGjvKUqXFOC+L7YegerkQzt9DbU+Lg6dIgskv7k3yOVnpevXLqx5e9SSVlH4O
KkbM2idmAw/0Oy23YNFnXN/FdlqcwjQ65I8Z+lxt92C3bNfJFtNuNidZ69OavEin
LwIDAQAB
-----END PUBLIC KEY-----



In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

# Load the private key
with open("D:\private_key.pem", "rb") as key_file:  # replace with your actual private key file path
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # replace with your password if the private key is encrypted
        backend=default_backend()
    )

# Assume that we have a message
message = "This is a message that needs to be signed"

# Sign the message
signature = private_key.sign(
    message.encode(),
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

print(signature)


b'K\x0f\xc6j)\x0e\xcf!P\xdc\xc1\x01\xeb\xf2\x96f=^A\xbaaU\xf54\x0e\x1f\xb6fP\x92\xb0l\xa6\x0e\xee\xd7\xc0iEG\xbb;f\xc1p\x0b\xd2\xa5\xb2W)gL\xde\xdd\r\xa0[\xe6\xfa\xc6VD\x97\xe8\xc7\xe6\x92\xe1\xa3:\xe4]k\xcb\x9dF^\x850\x95\x17\xc4s\xaa\x93\xb2}-j\xe6\xc3\xf7{\x8e\xadd\xc0\x14X\xb0\xb4\x9b1\xe8W\xd2[g\x82\xdd\x14\xd0\x81\\sZF\xfd.\xc6\x03\x8d\x0e\\\x92\'\x96X\x1fR\x94\xdc\x0b;\xc7\xff[\xfdl\x1e\x85\xcc\x1eQ\xcb\xa3\xd0\xb6/\x9a\x84Y\xbd\x9a\xb4\x8a\xf4\'5\x88\x88\xbd8,\x19\x03:\x16~\x84g\xe2\x0e\x1e\x9a\xe4\xd9\xa2\xbe+_\xf5jsC\x04\xd5>\xce\xf7\xcd\xabo\xa9\x92&\x93\x8a\x08\xce,\xad\xe9\x07\x96\x9c""\xed\xec\xe7f\x18\xc1\xc0\xbf\x95i"5D"y\xee%\xd2KD31c$5\x1f \xc6R/b;;m32\x18T\xc1\x12\x1b\xb0I=\x92\xc1P'


In [6]:
# Save the public key to a file
with open("D:\second_party_public_key.pem", "wb") as f:  # replace with your actual file path
    f.write(public_key_pem)

# Save the signature to a file
#with open("D:\signature.sig", "wb") as f:  # replace with your actual file path
#    f.write(signature)


In [4]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

# 读取公钥
with open('second_party_public_key.pem', 'r') as f:
    public_key = f.read()

# 读取私钥
with open('second_party_private_key.pem', 'r') as f:
    private_key = serialization.load_pem_private_key(
        f.read().encode(),
        password=None,
        backend=default_backend()
    )

# 用私钥对公钥进行签名
signer = private_key.sign(
    public_key.encode(),
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# 保存签名到文件
with open('second_party_signature.sig', 'wb') as f:
    f.write(signer)


In [5]:
%pip install pillow


You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
%pip install qrcode[pil]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [19]:
%pip install pycryptodome
%pip install pycrypto


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached pycrypto-2.6.1.tar.gz (446 kB)
Using legacy 'setup.py install' for pycrypto, since package 'wheel' is not installed.
    Running setup.py install for pycrypto: started
    Running setup.py install for pycrypto: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\think\AppData\Local\Programs\Python\Python39\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\think\\AppData\\Local\\Temp\\pip-install-5s9sj8y3\\pycrypto\\setup.py'"'"'; __file__='"'"'C:\\Users\\think\\AppData\\Local\\Temp\\pip-install-5s9sj8y3\\pycrypto\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\think\AppData\Local\Temp\pip-record-ms6fkvd9\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\Users\think\AppData\Local\Programs\Python\Python39\Include\pycrypto'
         cwd: C:\Users\think\AppData\Local\Temp\pip-install-5s9sj8y3\pycrypto\
    Complete output (153 lines):
    running install
    running build
    running build_py
    creating build
    creating build\lib.win-amd64-3.9
    

In [18]:
from Cryptodome.PublicKey import RSA
from Cryptodome.Signature import pkcs1_15
from Cryptodome.Hash import SHA256
import base64

# 生成三个公私钥对
key_pairs = [RSA.generate(2048) for _ in range(3)]

# 保存公钥和私钥到文件
for i, key_pair in enumerate(key_pairs, 1):
    # 保存公钥
    with open(f"public_key_{i}.pem", "wb") as pub_file:
        pub_file.write(key_pair.publickey().export_key())

    # 保存私钥
    with open(f"private_key_{i}.pem", "wb") as priv_file:
        priv_file.write(key_pair.export_key())

# 使用私钥生成三个签名，这里我们以公钥作为要签名的数据，并保存到文件
for i, key_pair in enumerate(key_pairs, 1):
    h = SHA256.new(key_pair.publickey().export_key())
    signature = pkcs1_15.new(key_pair).sign(h)
    encoded_signature = base64.b64encode(signature)

    with open(f"signature_{i}.txt", "w") as sig_file:
        sig_file.write(encoded_signature.decode())


ModuleNotFoundError: No module named 'Cryptodome'